<a href="https://colab.research.google.com/github/MtFeather/AI-and-Security/blob/master/pix2pixGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安裝所需軟體

In [10]:
!pip3 install pydot pydot-ng graphviz    # 模塊提供了畫出Keras模型的函數
!pip3 install ipython                    # Python是一種基於Python的交互式解釋器
!pip3 uninstall keras -y && pip3 install keras==2.2.1    # Keras 是一個用 Python 編寫的高級神經網絡 API

Uninstalling Keras-2.2.1:
  Successfully uninstalled Keras-2.2.1
  Using cached https://files.pythonhosted.org/packages/62/51/0192489a2614e8c6a22de860e43221e566f4bbd44a047ff48c2fdbc59373/Keras-2.2.1-py2.py3-none-any.whl


# 資料配置

In [11]:
!wget -N http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/cityscapes.tar.gz
!mkdir -p /data/cityscapes/
!tar -zxvf cityscapes.tar.gz -C /data/cityscapes/
!rm cityscapes.tar.gz
!mkdir -p /out

--2019-01-15 15:55:01--  http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/cityscapes.tar.gz
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.189.73
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103441232 (99M) [application/x-gzip]
Saving to: ‘cityscapes.tar.gz’

cityscapes.tar.gz   100%[===================>]  98.65M  2.96MB/s    in 47s     

2019-01-15 15:55:49 (2.08 MB/s) - ‘cityscapes.tar.gz’ saved [103441232/103441232]

cityscapes/
cityscapes/train/
cityscapes/train/17.jpg
cityscapes/train/234.jpg
cityscapes/train/2970.jpg
cityscapes/train/1.jpg
cityscapes/train/10.jpg
cityscapes/train/100.jpg
cityscapes/train/1000.jpg
cityscapes/train/1001.jpg
cityscapes/train/1002.jpg
cityscapes/train/1003.jpg
cityscapes/train/1004.jpg
cityscapes/train/1005.jpg
cityscapes/train/1006.jpg
cityscapes/train/1007.jpg
cityscapes/train/1008.jpg
cityscapes/train/

In [12]:
!ls /data/cityscapes/cityscapes/

train  val


# 下載資料模組
模組有cityscapes, night2day, edges2handbags, edges2shoes, facades, maps

In [13]:
!wget -N https://raw.githubusercontent.com/phillipi/pix2pix/master/datasets/download_dataset.sh -O /data/download_dataset.sh
!bash -E /data/download_dataset.sh maps

for details.

--2019-01-15 15:55:58--  https://raw.githubusercontent.com/phillipi/pix2pix/master/datasets/download_dataset.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 531 [text/plain]
Saving to: ‘/data/download_dataset.sh’

/data/download_data 100%[===================>]     531  --.-KB/s    in 0s      

2019-01-15 15:55:58 (82.4 MB/s) - ‘/data/download_dataset.sh’ saved [531/531]

Specified [maps]
for details.

--2019-01-15 15:56:00--  http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/maps.tar.gz
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.189.73
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 250242400 (239M) [

# generator.py

In [14]:
#!/usr/bin/env python
import sys    # sys模塊提供了一系列有關Python運行環境的變量和函數
import numpy as np    # 維度陣列與矩陣運算
from keras.layers import Dense, Reshape, Input, BatchNormalization, Concatenate    
from keras.layers.core import Activation    # 核心網路層
from keras.layers.convolutional import UpSampling2D, Convolution2D, MaxPooling2D,Deconvolution2D    # 捲積層模組
from keras.layers.advanced_activations import LeakyReLU    # 激活層模組
from keras.models import Sequential, Model    # 模型
from keras.optimizers import Adam, SGD, Nadam,Adamax   # 優化器
from keras import initializers    # 始化定義了設置Keras各層權重隨機初始值的方法
from keras.utils import plot_model  # 可視化模組

class Generator(object):    # 定義類別名稱為Generator
    def __init__(self, width = 28, height= 28, channels = 1):
        
        self.W = width    # 圖片寬度
        self.H = height    #圖片高度
        self.C = channels  #彩色
        self.SHAPE = (width,height,channels)    #型態

        self.Generator = self.model()    # 執行底下model模組
        #Adam優化器(學習率,0 <beta <1通常接近於1, 每次參數更新後學習率衰減值)
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5,decay=1e-5)
        # 配置訓練模型(損失函數, 優化器名稱, 訓練和測試期間的模型評估標準)
        self.Generator.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER,metrics=['accuracy'])    

        self.save_model()    # 執行底下save_model模組
        self.summary()    # 執行底下summary模組

    def model(self):
        input_layer = Input(shape=self.SHAPE)   # 放入型態
        # 二維卷積層(卷積核的數目, 單個整數或由兩個整數構成的列表/元組, 步幅, same代表保留邊界處的捲積結果, 激活函數)
        down_1 = Convolution2D(64  , kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(input_layer)    

        down_2 = Convolution2D(64*2, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(down_1)
        # 批標準化
        norm_2 = BatchNormalization()(down_2)

        down_3 = Convolution2D(64*4, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_2)
        norm_3 = BatchNormalization()(down_3)

        down_4 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_3)
        norm_4 = BatchNormalization()(down_4)

        down_5 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_4)
        norm_5 = BatchNormalization()(down_5)

        down_6 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_5)
        norm_6 = BatchNormalization()(down_6)

        down_7 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_6)
        norm_7 = BatchNormalization()(down_7)

        # 2D輸入的上採樣層(size：整數，或2個整數的元組。行和列的上採樣因子)
        upsample_1 = UpSampling2D(size=2)(norm_7)
        up_conv_1 = Convolution2D(64*8, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_1)
        # 批標準化(動態均值的動量)
        norm_up_1 = BatchNormalization(momentum=0.8)(up_conv_1)
        # 連接一個輸入張量的列表
        add_skip_1 = Concatenate()([norm_up_1,norm_6])


        upsample_2 = UpSampling2D(size=2)(add_skip_1)
        up_conv_2 = Convolution2D(64*8, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_2)
        norm_up_2 = BatchNormalization(momentum=0.8)(up_conv_2)
        add_skip_2 = Concatenate()([norm_up_2,norm_5])

        upsample_3 = UpSampling2D(size=2)(add_skip_2)
        up_conv_3 = Convolution2D(64*8, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_3)
        norm_up_3 = BatchNormalization(momentum=0.8)(up_conv_3)
        add_skip_3 = Concatenate()([norm_up_3,norm_4])


        upsample_4 = UpSampling2D(size=2)(add_skip_3)
        up_conv_4 = Convolution2D(64*4, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_4)
        norm_up_4 = BatchNormalization(momentum=0.8)(up_conv_4)
        add_skip_4 = Concatenate()([norm_up_4,norm_3])

        upsample_5 = UpSampling2D(size=2)(add_skip_4)
        up_conv_5 = Convolution2D(64*2, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_5)
        norm_up_5 = BatchNormalization(momentum=0.8)(up_conv_5)
        add_skip_5 = Concatenate()([norm_up_5,norm_2])

        upsample_6 = UpSampling2D(size=2)(add_skip_5)
        up_conv_6 = Convolution2D(64, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_6)
        norm_up_6 = BatchNormalization(momentum=0.8)(up_conv_6)
        add_skip_6 = Concatenate()([norm_up_6,down_1])

        last_upsample = UpSampling2D(size=2)(add_skip_6)
        output_layer = Convolution2D(self.C, kernel_size=4, strides=1, padding='same',activation='tanh')(last_upsample)
        
        return Model(input_layer,output_layer)

    def summary(self):
        return self.Generator.summary()

    def save_model(self):
        plot_model(self.Generator, to_file='/out/Generator_Model.png')

Using TensorFlow backend.


# discriminator.py

In [0]:
#!/usr/bin/env python3
import sys    # sys模塊提供了一系列有關Python運行環境的變量和函數
import numpy as np    # 維度陣列與矩陣運算
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Lambda, Concatenate    
from keras.layers.core import Activation    # 核心網路層
from keras.layers.convolutional import Convolution2D    # 捲積層模組
from keras.layers.advanced_activations import LeakyReLU    # 激活層模組
from keras.models import Sequential, Model    # 模型
from keras.optimizers import Adam, SGD,Nadam, Adamax    # 優化器
import keras.backend as K    # 提供了快速構建深度學習網絡的模塊.Keras並不處理如張量乘法，卷積等底層操作
from keras.utils import plot_model    # 可視化模型


class Discriminator(object):
    def __init__(self, width = 28, height= 28, channels = 1, starting_filters=64):
        self.W = width    # 圖片寬度
        self.H = height    # 圖片高度
        self.C = channels    # 彩色
        self.CAPACITY = width*height*channels    
        self.SHAPE = (width,height,channels)
        self.FS = starting_filters #FilterStart
        
        self.Discriminator = self.model()    # 執行底下model模組
        # Adam優化器(學習率,0 <beta <1通常接近於1, 每次參數更新後學習率衰減值)
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5,decay=1e-5)    
        # 配置訓練模型(損失函數, 優化器名稱, 訓練和測試期間的模型評估標準)
        self.Discriminator.compile(loss='mse', optimizer=self.OPTIMIZER, metrics=['accuracy'] )

        self.save_model()    # 執行底下save_model模組
        self.summary()    # 執行底下save_model模組

    def model(self):


        input_A = Input(shape=self.SHAPE)
        input_B = Input(shape=self.SHAPE)
        # 連接一個輸入張量的列表(axis :連接的軸)
        input_layer = Concatenate(axis=-1)([input_A, input_B])
        
        # 二維卷積層(卷積核的數目, 單個整數或由兩個整數構成的列表/元組, 步幅, same代表保留邊界處的捲積結果, 激活函數)
        up_layer_1 = Convolution2D(self.FS, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(input_layer)

        up_layer_2 = Convolution2D(self.FS*2, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(up_layer_1)
        # 批標準化(動態均值的動量)
        leaky_layer_2 =  BatchNormalization(momentum=0.8)(up_layer_2)

        up_layer_3 = Convolution2D(self.FS*4, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(leaky_layer_2)
        leaky_layer_3 =  BatchNormalization(momentum=0.8)(up_layer_3)

        up_layer_4 = Convolution2D(self.FS*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(leaky_layer_3)
        leaky_layer_4 = BatchNormalization(momentum=0.8)(up_layer_4)

        output_layer = Convolution2D(1, kernel_size=4, strides=1, padding='same')(leaky_layer_4)
        
        return Model([input_A, input_B],output_layer)

    def summary(self):
        return self.Discriminator.summary()

    def save_model(self):
        plot_model(self.Discriminator, to_file='/out/Discriminator_Model.png')

# gan.py

In [0]:
#!/usr/bin/env python3
import sys
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input
from keras.optimizers import Adam, SGD
from keras.utils import plot_model

class GAN(object):
    def __init__(self, model_inputs=[],model_outputs=[]):
        # 隨機梯度下降優化器(學習率, 動量參數)
        self.OPTIMIZER = SGD(lr=2e-4,nesterov=True)

        self.inputs = model_inputs  
        self.outputs = model_outputs
        self.gan_model = Model(inputs = self.inputs, outputs = self.outputs)
        # Adam優化器(學習率,0 <beta <1通常接近於1)
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5)
        # 配置訓練模型(損失函數,loss_weights :指定標量係數（Python浮點數）的列表或字典，用以衡量損失函數對不同的模型輸出的貢獻, 優化器名稱)
        self.gan_model.compile(loss=['mse', 'mae'],
                            loss_weights=[  1, 100],
                            optimizer=self.OPTIMIZER)
        self.save_model()
        self.summary()

    def model(self):
        model = Model()
        return model

    def summary(self):
        return self.gan_model.summary()

    def save_model(self):
        plot_model(self.gan_model, to_file='/out/GAN_Model.png')


# train.py

In [0]:
#!/usr/bin/env python3
from keras.layers import Input
from keras.datasets import mnist    # 匯入MNIST 手寫字符數據集
from random import randint    # 整數亂數產生
import numpy as np   # # 維度陣列與矩陣運算 
import matplotlib.pyplot as plt    # 資料視覺化
from copy import deepcopy    # 匯入身拷貝
import os    #os 模塊提供了非常豐富的方法用來處理文件和目錄
from PIL import Image    # 影像處理套件
import random
import numpy as np

class Trainer:
    def __init__(self, height = 256, width = 256, channels=3, epochs = 50000, batch = 1, checkpoint = 50, train_data_path = '',test_data_path=''):
        self.EPOCHS = epochs
        self.BATCH = batch
        self.H = height
        self.W = width
        self.C = channels
        self.CHECKPOINT = checkpoint

        # Read in the dataset for training and another set of dataset for comparing when we use the checkpoint plots:
        self.X_train_B, self.X_train_A = self.load_data(train_data_path)
        self.X_test_B, self.X_test_A  = self.load_data(test_data_path)

        # Instantiate the Generator object within the training class:
        self.generator = Generator(height=self.H, width=self.W, channels=self.C)

        # Create two input with the shape of our input images—in this case, our images have the exact same size, shape, and number of channels:
        self.orig_A = Input(shape=(self.W, self.H, self.C))
        self.orig_B = Input(shape=(self.W, self.H, self.C))
        
        # Here‘s the conditional training part—we use orig_b as the input to the generator:
        self.fake_A = self.generator.Generator(self.orig_B)

        # Next, we build the Discriminator object and set the discriminator network trainable to False:
        self.discriminator = Discriminator(height=self.H, width=self.W, channels=self.C)
        self.discriminator.trainable = False
        # With this discriminator object, we then point the inputs to the fake_A and orig_B networks we created:
        self.valid = self.discriminator.Discriminator([self.fake_A,self.orig_B])

        # Finally, we have all the correct connections setup. We now create our adversarial model with the inputs of orig_A and orig_B and the outputs of valid and fake_A:
        model_inputs  = [self.orig_A,self.orig_B]
        model_outputs = [self.valid, self.fake_A]
        self.gan = GAN(model_inputs=model_inputs,model_outputs=model_outputs)
        
        
    # The load data function helps define the downloaded input data—the images are stitched together as 256 x 512, so this function reads them in and splits them into two arrays: 
    def load_data(self,data_path):
        listOFFiles = self.grabListOfFiles(data_path,extension="jpg")
        imgs_temp = np.array(self.grabArrayOfImages(listOFFiles))
        imgs_A = []
        imgs_B = []
        for img in imgs_temp:
            imgs_A.append(img[:,:self.H])
            imgs_B.append(img[:,self.H:])

        imgs_A_out = self.norm_and_expand(np.array(imgs_A))
        imgs_B_out = self.norm_and_expand(np.array(imgs_B))

        return imgs_A_out, imgs_B_out

    # This is a convenience function that puts the array in the correct shape for the network to use it:
    def norm_and_expand(self,arr):
        arr = (arr.astype(np.float32) - 127.5)/127.5
        normed = np.expand_dims(arr, axis=3)
        return normed

    # This function lets us grab a list of files from a starting directory:
    def grabListOfFiles(self,startingDirectory,extension=".webp"):
        listOfFiles = []
        for file in os.listdir(startingDirectory):
            if file.endswith(extension):
                listOfFiles.append(os.path.join(startingDirectory, file))
        return listOfFiles

    # Given a list of files, read in the images into an array and return them:
    def grabArrayOfImages(self,listOfFiles,gray=False):
        imageArr = []
        for f in listOfFiles:
            if gray:
                im = Image.open(f).convert("L")
            else:
                im = Image.open(f).convert("RGB")
            imData = np.asarray(im)
            imageArr.append(imData)
        return imageArr


    def train(self):
        # First, we define a train method that will run through the number of epochs—at each epoch, you will want to make a copy of the A and B training data:
        for e in range(self.EPOCHS):
            b = 0
            X_train_A_temp = deepcopy(self.X_train_A)
            X_train_B_temp = deepcopy(self.X_train_B)
            
            # In the next step, we define the number of batches (batch size of 1 for style transfer) and run through the batches:
            number_of_batches = len(self.X_train_A)
        
            for b in range(number_of_batches):
                # In this step, we are grabbing random indices for both A and B sets (the data is paired so we grab the same indices for each of the A and B datasets):
                # Train Discriminator
                # Grab Real Images for this training batch
                starting_ind = randint(0, (len(X_train_A_temp)-1))
                real_images_raw_A = X_train_A_temp[ starting_ind : (starting_ind + 1) ]
                real_images_raw_B = X_train_B_temp[ starting_ind : (starting_ind + 1) ]
                
                # After grabbing our images, we delete the images from the temp arrays:
                # Delete the images used until we have none left
                X_train_A_temp = np.delete(X_train_A_temp,range(starting_ind,(starting_ind + 1)),0)
                X_train_B_temp = np.delete(X_train_B_temp,range(starting_ind,(starting_ind + 1)),0)
                
                # We create our batches for the our prediction and training steps:
                batch_A = real_images_raw_A.reshape( 1, self.W, self.H, self.C )
                batch_B = real_images_raw_B.reshape( 1, self.W, self.H, self.C )

                # Using the PatchGAN papers structure to the Y labels, we create the y_valid and y_fake labels for our training:
                # PatchGAN
                y_valid = np.ones((1,)+(int(self.W / 2**4), int(self.W / 2**4), 1))
                y_fake = np.zeros((1,)+(int(self.W / 2**4), int(self.W / 2**4), 1))
                
                # Next, we create our Fake A images by using the generator to generate results based on the batch B input:
                fake_A = self.generator.Generator.predict(batch_B)

                # We train our discriminator on both the real and fake datasets
                # Now, train the discriminator with this batch of reals
                discriminator_loss_real = self.discriminator.Discriminator.train_on_batch([batch_A,batch_B],y_valid)[0]
                discriminator_loss_fake = self.discriminator.Discriminator.train_on_batch([fake_A,batch_B],y_fake)[0]
                full_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

                # In this step, we compute an aggregate loss by just averaging the two values:
                generator_loss = self.gan.gan_model.train_on_batch([batch_A, batch_B],[y_valid,batch_A])    

                # In the last step, we print diagnostics at every batch and every epoch
                print ('Batch: '+str(int(b))+', [Full Discriminator :: Loss: '+str(full_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                if b % self.CHECKPOINT == 0 :
                    label = str(e)+'_'+str(b)
                    self.plot_checkpoint(label)

            print ('Epoch: '+str(int(e))+', [Full Discriminator :: Loss:'+str(full_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                        
        return

    def plot_checkpoint(self,b):
        orig_filename = "/out/batch_check_"+str(b)+"_original.png"

        # Next, we define how many rows and columns we'll need in our graph plus the number of samples we would like to grab for checkpoint evaluation of the models:
        r, c = 3, 3
        random_inds = random.sample(range(len(self.X_test_A)),3)
        
        #Next, we grab images from our test set and format them to evaluated by our generator:
        imgs_A = self.X_test_A[random_inds].reshape(3, self.W, self.H, self.C )
        imgs_B = self.X_test_B[random_inds].reshape( 3, self.W, self.H, self.C )
        fake_A = self.generator.Generator.predict(imgs_B)

        # We put all the data into a single array to make plotting easier:
        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Next, we rescale all of the images to be from 0-1 for plotting:
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        # In this step, we create the titles, configure the plot, and plot each image in the plot:
        titles = ['Style', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
              
        # In the last step, we save the figure and close the figure (saves memory on the machine):
        fig.savefig("/out/batch_check_"+str(b)+".png")
        plt.close('all')

        return

# run.py

In [0]:
#!/usr/bin/env python3

# Command Line Argument Method
HEIGHT  = 256
WIDTH   = 256
CHANNELS = 3
EPOCHS = 100
BATCH = 1
CHECKPOINT = 50
TRAIN_PATH = "/data/cityscapes/cityscapes/train/"
TEST_PATH = "/data/cityscapes/cityscapes/val/"

trainer = Trainer(height=HEIGHT,width=WIDTH, channels=CHANNELS,epochs =EPOCHS,\
                 batch=BATCH,\
                 checkpoint=CHECKPOINT,\
                 train_data_path=TRAIN_PATH,\
                 test_data_path=TEST_PATH)
trainer.train()

/usr/local/lib/python3.6/dist-packages/keras/activations.py:197: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 128, 128, 64) 3136        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 64, 64, 128)  131200      conv2d_15[0][0]                  
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 64, 64, 128)  512         conv2d_16[0][0]                  
__________________________________________________________________________________________________
conv2d_17 

/usr/local/lib/python3.6/dist-packages/keras/activations.py:197: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
concatenate_13 (Concatenate)    (None, 256, 256, 6)  0           input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 128, 128, 64) 6208        concatenate_13[0][0]             
__________